In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [2]:
data = pd.read_csv("master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features


data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 
                  'polarity_category', 'overall_content', 'polarity_category_Neutral' , 'polarity_category_Positive',
                  'text_average_word_length',  'title_average_word_length',  'text_punctuation_count',
                    'title_punctuation_count',  'text_stopwords_count',  'title_stopwords_count',], axis=1)

#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity'

In [3]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [4]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'flesch_readability',
 'subjectivity',
 'polarity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability']

In [5]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,0.006674,0.402753,0.056997,0.190953,0.324903,0.125233,0.185360,0.173552
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.003826,0.124806,0.105170,0.281023,0.312565,0.210573,0.258829,0.234861
min,0.000000,1.000000,1.000000,1.000000,1.000000,-83.616811,0.000000,-1.000000,0.000376,0.000246,0.000120,0.000303,0.000268
25%,0.000000,220.000000,10.000000,7.000000,1.000000,-0.169506,0.337127,0.000000,0.002949,0.010969,0.001305,0.003113,0.003024
50%,0.000000,376.000000,11.000000,13.000000,1.000000,0.050549,0.405745,0.054757,0.030471,0.231605,0.006765,0.056382,0.056373
75%,1.000000,522.000000,14.000000,20.000000,1.000000,0.267479,0.475000,0.108144,0.282926,0.584998,0.157612,0.272065,0.268291
max,1.000000,8436.000000,45.000000,321.000000,4.000000,2.614284,1.000000,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767


In [6]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 4222)


x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = undersample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()

0    10477
1    10477
Name: class, dtype: int64

In [7]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

scaler = MinMaxScaler()

# Since polarity_category_Neutral and polarity_category_Positive are already from 0 to 1 scale, we will not have to further normalize it.
cols = data.iloc[:,0:-1].columns

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
x_train[cols] = scaler.fit_transform(x_train[cols]) 

x_test[cols] = scaler.transform(x_test[cols])
x_validation[cols] = scaler.transform(x_validation[cols])



In [8]:
x_train.describe()

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000
mean,0.048785,0.259481,0.043826,0.024387,0.971038,0.407669,0.528753,0.177295,0.340064,0.125128,0.180252,0.178561
std,0.038376,0.088568,0.037013,0.093790,0.011930,0.125005,0.053675,0.273024,0.314611,0.209302,0.254787,0.236632
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.026793,0.204545,0.018750,0.000000,0.968794,0.341341,0.500000,0.002518,0.026349,0.001126,0.002680,0.002849
50%,0.044458,0.250000,0.037500,0.000000,0.971558,0.410979,0.527639,0.022932,0.261797,0.007950,0.053078,0.062767
75%,0.061648,0.318182,0.059375,0.000000,0.974295,0.479956,0.554395,0.243779,0.606549,0.159221,0.262408,0.281911
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Linear SVM

In [9]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=4222)
linearSVC.fit(x_train, np.ravel(y_train))
y_predval_linearSVC = linearSVC.predict(x_validation)

#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_linearSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_linearSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_linearSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_linearSVC))
print("-------------------------------")


#test metrics
y_pred_linearSVC = linearSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))

Performance on Validation set:
Accuracy: 0.9071391619244698
Precision: 0.8986758779504893
Recall: 0.8950688073394495
F1_score: 0.8968687158862396
-------------------------------
Performance on  Test set:
Accuracy: 0.903388515261252
Precision: 0.8994770482277745
Recall: 0.8853302830997998
F1_score: 0.8923476005188068


In [10]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state=4222), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 10, 'gamma': 1}


In [12]:
finalSVC = SVC( kernel= 'linear', C= 10, gamma= 1, random_state = 4222)
finalSVC.fit(x_train, np.ravel(y_train))
y_predval_finalSVC = finalSVC.predict(x_validation)


#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_finalSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_finalSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_finalSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_finalSVC))
print("-------------------------------")


#test metrics
y_predtest_linearSVC = finalSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_predtest_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_predtest_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_predtest_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_predtest_linearSVC))

Performance on Validation set:
Accuracy: 0.9110191412312467
Precision: 0.9011461318051576
Recall: 0.9016628440366973
F1_score: 0.9014044138721697
-------------------------------
Performance on  Test set:
Accuracy: 0.9085618210036213
Precision: 0.902946273830156
Recall: 0.8939090649127824
F1_score: 0.8984049432389711
